In [1]:
#import necessary packages
from pathlib import Path
import os
import ast
import shutil
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import LocalOutlierFactor
from sktime.transformations.panel.rocket import (
    MiniRocket,
    MiniRocketMultivariate,
    MiniRocketMultivariateVariable,
)

In [2]:
#collection of dataset information for each telemetry data 
training_dfs = []
test_dfs = []

training_df = []
test_df = []

file_names = []

label_df = pd.read_csv("labeled_anomalies.csv")

train_data_path = "npy_train" 
test_data_path = "npy_test" 

os.makedirs("raw_train", exist_ok=True)
os.makedirs("raw_test", exist_ok=True)

index = 0
for root, _, files in os.walk(train_data_path):
    for file in files:
        example_path = os.path.join(root, file)
        filename = Path(example_path).stem
        
        row_indices = label_df[label_df['chan_id'] == filename].index.tolist()
        if not row_indices:
            continue
        spacecraft = label_df.loc[row_indices[0], 'spacecraft']
        
        if (spacecraft == "SMAP"): 
            data = np.load(example_path)
            df = pd.DataFrame(data)
            print("(train) df: ", filename, df.shape, "index: ", index)
            df.to_csv(f"raw_train/{filename}.csv", index=False)
            training_dfs.append(df) 
            index += 1

index = 0
for root, _, files in os.walk(test_data_path):
    for file in files:
        example_path = os.path.join(root, file)
        filename = Path(example_path).stem
        
        row_indices = label_df[label_df['chan_id'] == filename].index.tolist()
        if not row_indices:
            continue
        spacecraft = label_df.loc[row_indices[0], 'spacecraft']
        
        if (spacecraft == "SMAP"): 
            data = np.load(example_path)
            df = pd.DataFrame(data)
            print("(test) df: ", filename, df.shape, "index: ", index)
            df.to_csv(f"raw_test/{filename}.csv", index=False)
            test_dfs.append(df) 
            file_names.append(filename)
            
            index += 1

In [2]:
def get_data():
    training_dfs = {}
    test_dfs = {}
    file_names = {"train": {}, "test": {}}
    label_df = pd.read_csv("labeled_anomalies.csv")
    train_data_path = "raw_train" 
    test_data_path = "raw_test" 
    
    for root, _, files in os.walk(train_data_path):
        for file in files:
            example_path = os.path.join(root, file)
            filename = Path(example_path).stem
            df = pd.read_csv(example_path)
            channel = filename[0]            
            if (training_dfs.get(channel) == None):
                training_dfs[channel] = []
                file_names["train"][channel] = []
                training_dfs[channel].append(df)
                file_names["train"][channel].append(filename)
            else:
                training_dfs[channel].append(df)
                file_names["train"][channel].append(filename)
    
    for root, _, files in os.walk(test_data_path):
        for file in files:
            example_path = os.path.join(root, file)
            filename = Path(example_path).stem
            df = pd.read_csv(example_path)
            channel = filename[0]
            if (test_dfs.get(channel) == None):
                test_dfs[channel] = []
                file_names["test"][channel] = []
                test_dfs[channel].append(df)
                file_names["test"][channel].append(filename)
            else:
                test_dfs[channel].append(df)
                file_names["test"][channel].append(filename)

    return (training_dfs, test_dfs, label_df, file_names)

In [3]:
def create_windows(training_dfs, test_dfs, window_size, window_overlap, label_df, file_names):
    training_data = {}
    test_data = {}
    label_data = {}
    
    for channel in training_dfs.keys():
        training_data[channel] = []
        for df in training_dfs[channel]: 
            for i in range(0, len(df), window_overlap):
                window = []
                if i + window_size > len(df): 
                    window = df.iloc[-window_size:].to_numpy().tolist()
                else: 
                    window = df.iloc[i:i + window_size].to_numpy().tolist()
                np_window = np.array(window)
                transposed_window = np_window.T
                normal_window = transposed_window.tolist() 
                training_data[channel].append(normal_window)
                
    for channel in test_dfs.keys():
        test_data[channel] = []
        label_data[channel] = []
        for i in range(len(test_dfs[channel])):
            df = test_dfs[channel][i]
            for j in range(0, len(df), window_overlap):
                window = []
                if j + window_size > len(df): 
                    window = df.iloc[-window_size:].to_numpy().tolist()
                else: 
                    window = df.iloc[j:j + window_size].to_numpy().tolist()
                row_indices = label_df[label_df["chan_id"] == file_names["test"][channel][i]].index.tolist()
                
                if not row_indices:
                    continue

                anomaly_sequence = label_df.loc[row_indices[0], 'anomaly_sequences']
                anomaly_sequence = ast.literal_eval(anomaly_sequence)
                labeled = False
                for anomalies in anomaly_sequence:
                    if (not(anomalies[1] <= j or anomalies[0] >= j + window_size)) and labeled == False:
                        label_data[channel].append(1)
                        labeled = True
                if labeled == False:
                    label_data[channel].append(0)
                np_window = np.array(window)
                transposed_window = np_window.T
                normal_window = transposed_window.tolist() 
                test_data[channel].append(normal_window)
                
    return (training_data, test_data, label_data)

In [9]:
window_size = 60 
window_overlap = 20
training_dfs, test_dfs, label_df, file_names = get_data()

X_train_collection, X_test_collection, y_test_collection = create_windows(training_dfs, test_dfs, window_size, window_overlap, label_df, file_names)

data = {}
os.makedirs("data", exist_ok=True)

for channel in X_train_collection:
    if data.get(channel) is None:
        data[channel] = {}
    if data.get(channel).get("train") is None:
        X_train = np.array(X_train_collection[channel])
        path = f"data/{channel}/train"
        os.makedirs(path, exist_ok=True)
        np.save(path + f"/{channel}.npy", X_train)
        data[channel]["train"] = path + f"/{channel}.npy"
        print("train: ", channel, "shape: ", X_train.shape)
    else:
        print("Error: duplicate training data sets")
   
    
for channel in X_test_collection: 
    if data.get(channel) is None:
        continue
    if data.get(channel).get("test") is None:
        X_test = np.array(X_test_collection[channel])
        path = f"data/{channel}/test"
        os.makedirs(path, exist_ok=True)
        np.save(path + f"/{channel}.npy", X_test)
        data[channel]["test"] = path + f"/{channel}.npy"
        print("test: ", channel, "shape: ", X_test.shape)
    else:
        print("Error: duplicate testing data sets")
   
  
for channel in y_test_collection:
    if data.get(channel) is None:
        continue
    if data.get(channel).get("label") is None:
        y_test = np.array(y_test_collection[channel])
        path = f"data/{channel}/label"
        os.makedirs(path, exist_ok=True)
        np.save(path + f"/{channel}.npy", y_test)
        data[channel]["label"] = path + f"/{channel}.npy"
        print("label: ", channel, "shape: ", y_test.shape)
    else:
        print("Error: duplicate label data sets")
    

train:  A shape:  (842, 25, 60)
train:  P shape:  (703, 25, 60)
train:  F shape:  (432, 25, 60)
train:  D shape:  (1445, 25, 60)
train:  R shape:  (144, 25, 60)
train:  G shape:  (793, 25, 60)
train:  T shape:  (431, 25, 60)
train:  E shape:  (1867, 25, 60)
train:  S shape:  (141, 25, 60)
train:  B shape:  (122, 25, 60)
test:  E shape:  (5486, 25, 60)
test:  T shape:  (1292, 25, 60)
test:  G shape:  (2405, 25, 60)
test:  D shape:  (4789, 25, 60)
test:  A shape:  (3374, 25, 60)
test:  F shape:  (1281, 25, 60)
test:  P shape:  (2056, 25, 60)
test:  B shape:  (403, 25, 60)
test:  R shape:  (363, 25, 60)
test:  S shape:  (367, 25, 60)
label:  E shape:  (5486,)
label:  T shape:  (1292,)
label:  G shape:  (2405,)
label:  D shape:  (4789,)
label:  A shape:  (3374,)
label:  F shape:  (1281,)
label:  P shape:  (2056,)
label:  B shape:  (403,)
label:  R shape:  (363,)
label:  S shape:  (367,)


In [5]:
channel_names = []

for channel in os.listdir("data"):
    if os.path.isdir(os.path.join("data", channel)):
        channel_names.append(channel)
print(channel_names)

['T', 'G', 'S', 'P', 'D', 'F', 'R', 'A', 'E', 'B']


In [6]:
transformed_data= {}
for channel in channel_names:
    transformed_data[channel] = {}
    training_data_path = f"data/{channel}/train/{channel}.npy"
    testing_data_path = f"data/{channel}/test/{channel}.npy"
    label_data_path = f"data/{channel}/label/{channel}.npy"
    
    X_train = np.load(training_data_path)
    X_test = np.load(testing_data_path)
    y_test = np.load(label_data_path)

    minirocket = MiniRocketMultivariate(n_jobs = 2, random_state = 42) 
    minirocket.fit(X_train)
    X_transform_train = minirocket.transform(X_train)
    X_transform_test = minirocket.transform(X_test)

    transformed_data[channel]["train"] = X_transform_train
    print("train: ", channel, "shape: ", X_transform_train.shape)
    transformed_data[channel]["test"] = X_transform_test
    print("test: ", channel, "shape: ", X_transform_test.shape)
    transformed_data[channel]["label"] = y_test
    print("label: ", channel, "shape: ", y_test.shape)

/home/DAVIDSON/haotaki/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:373: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


train:  T shape:  (431, 9996)
test:  T shape:  (1292, 9996)
label:  T shape:  (1292,)
train:  G shape:  (793, 9996)
test:  G shape:  (2405, 9996)
label:  G shape:  (2405,)
train:  S shape:  (141, 9996)
test:  S shape:  (367, 9996)
label:  S shape:  (367,)
train:  P shape:  (703, 9996)
test:  P shape:  (2056, 9996)
label:  P shape:  (2056,)
train:  D shape:  (1445, 9996)
test:  D shape:  (4789, 9996)
label:  D shape:  (4789,)
train:  F shape:  (432, 9996)
test:  F shape:  (1281, 9996)
label:  F shape:  (1281,)
train:  R shape:  (144, 9996)
test:  R shape:  (363, 9996)
label:  R shape:  (363,)
train:  A shape:  (842, 9996)
test:  A shape:  (3374, 9996)
label:  A shape:  (3374,)
train:  E shape:  (1867, 9996)
test:  E shape:  (5486, 9996)
label:  E shape:  (5486,)
train:  B shape:  (122, 9996)
test:  B shape:  (403, 9996)
label:  B shape:  (403,)


In [7]:
fitted_data = {}

for channel in transformed_data: 
    fitted_data[channel] = {}
    X_transform_train = transformed_data[channel]["train"]
    X_transform_test = transformed_data[channel]["test"]
    scaler = MinMaxScaler()
    scaler.fit(X_transform_train)
    X_fit_train = scaler.transform(X_transform_train)
    X_fit_test = scaler.transform(X_transform_test)
    
    fitted_data[channel]["train"] = X_fit_train
    fitted_data[channel]["test"] = X_fit_test
    fitted_data[channel]["label"] = transformed_data[channel]["label"]
    print("train: ", channel, "shape: ", X_fit_train.shape)
    print("test: ", channel, "shape: ", X_fit_test.shape)
    print("label: ", channel, "shape: ", transformed_data[channel]["label"].shape)

train:  T shape:  (431, 9996)
test:  T shape:  (1292, 9996)
label:  T shape:  (1292,)
train:  G shape:  (793, 9996)
test:  G shape:  (2405, 9996)
label:  G shape:  (2405,)
train:  S shape:  (141, 9996)
test:  S shape:  (367, 9996)
label:  S shape:  (367,)
train:  P shape:  (703, 9996)
test:  P shape:  (2056, 9996)
label:  P shape:  (2056,)
train:  D shape:  (1445, 9996)
test:  D shape:  (4789, 9996)
label:  D shape:  (4789,)
train:  F shape:  (432, 9996)
test:  F shape:  (1281, 9996)
label:  F shape:  (1281,)
train:  R shape:  (144, 9996)
test:  R shape:  (363, 9996)
label:  R shape:  (363,)
train:  A shape:  (842, 9996)
test:  A shape:  (3374, 9996)
label:  A shape:  (3374,)
train:  E shape:  (1867, 9996)
test:  E shape:  (5486, 9996)
label:  E shape:  (5486,)
train:  B shape:  (122, 9996)
test:  B shape:  (403, 9996)
label:  B shape:  (403,)


In [65]:
# performance bad for channels B, R, and S due to their small training data size
# nu - controls boundary tightness (Smaller nu → tighter boundary → fewer false positives, Larger nu → looser boundary → higher recall)
# threshold	- converts scores (anomaly / normal)

# Large (1000+ windows/channel)	Mostly clean	0.01 – 0.02
# Medium (hundreds of windows)	Mostly clean	0.02 – 0.05
# Small or noisy	Some anomalies in training	0.05 – 0.1
#tighter nu makes more sense as the training data is supposed to have only normal data - needs confirmation at this point in time 

anomaly_rate = []
for channel in transformed_data: 
    anomaly_count = 0
    for label in fitted_data[channel]["label"]:
        if label == 1:
            anomaly_count += 1
    anomaly_rate.append(anomaly_count / len(fitted_data[channel]["label"]))
    
print(anomaly_rate)


[0.14628482972136223, 0.021205821205821207, 0.0681198910081744, 0.14348249027237353, 0.22635205679682607, 0.12724434035909446, 0.01928374655647383, 0.16478956727919383, 0.11009843237331389, 0.01488833746898263]


In [8]:
def predict(threshold, decision_scores):
    y_pred = []
    for score in decision_scores: 
        if score < threshold: 
            y_pred.append(1)
        else:
            y_pred.append(0)
    return np.array(y_pred)

In [9]:
def one_class_svm(channel_data, channel):
    print("channel:", channel)
    clf = OneClassSVM().fit(channel_data["train"])
    decision_scores = clf.decision_function(channel_data["test"])
    
    thresholds = []
    for score in decision_scores: 
        if score < 0:
            thresholds.append(score)
    
    best_f1_score = 0
    best_recall_score = 0
    best_thresholds = []
    best_threshold = 0

    # for th in thresholds:
    #     y_pred = predict(th, decision_scores)
    #     f1 = f1_score(channel_data["label"], y_pred)
    #     if (best_f1_score <= f1):
    #         best_f1_score = f1
    #         best_threshold = th

    for th in thresholds:
        y_pred = predict(th, decision_scores)
        f1 = f1_score(channel_data["label"], y_pred)
        recall = recall_score(channel_data["label"], y_pred)
        if (best_recall_score <= recall):
            best_recall_score = recall
            best_thresholds.append(th) 
    
    for th in best_thresholds:
        y_pred = predict(th, decision_scores)
        f1 = f1_score(channel_data["label"], y_pred)
        if (best_f1_score <= f1):
            best_f1_score = f1
            best_threshold = th
    
    y_test = channel_data["label"]
    y_pred = predict(best_threshold, decision_scores)
    print("best_threshold: ", best_threshold)
    print(classification_report(y_test, y_pred))
    precision = precision_score(y_test, y_pred, average='weighted')
    print("FDR:", 1 - precision)
    c_m = confusion_matrix(y_test, y_pred)
    print(c_m)

In [10]:
for channel in fitted_data:
    one_class_svm(fitted_data[channel], channel)

channel: T
best_threshold:  -0.027457979179303038
              precision    recall  f1-score   support

           0       0.82      0.59      0.68      1103
           1       0.09      0.25      0.14       189

    accuracy                           0.54      1292
   macro avg       0.46      0.42      0.41      1292
weighted avg       0.71      0.54      0.60      1292

FDR: 0.2864852626864265
[[646 457]
 [142  47]]
channel: G
best_threshold:  -5.657644072881624
              precision    recall  f1-score   support

           0       0.98      0.76      0.86      2354
           1       0.04      0.45      0.07        51

    accuracy                           0.76      2405
   macro avg       0.51      0.61      0.47      2405
weighted avg       0.96      0.76      0.84      2405

FDR: 0.03535299951608539
[[1800  554]
 [  28   23]]
channel: S
best_threshold:  -0.2859636559503791
              precision    recall  f1-score   support

           0       0.98      0.56      0.71    

In [11]:
def predict(threshold, decision_scores):
    y_pred = []
    for score in decision_scores: 
        if score < threshold: 
            y_pred.append(1)
        else:
            y_pred.append(0)
    return np.array(y_pred)

In [32]:
def isolation_forest_default(channel_data, channel):
    print("channel:", channel)
    #contamination=0
    clf = IsolationForest(contamination=0.5, random_state=42, n_jobs=-1).fit(channel_data["train"])
    # decision_scores = clf.decision_function(channel_data["test"])
    
    # thresholds = []
    # for score in decision_scores: 
    #     if score < 0:
    #         thresholds.append(score)
    
    # best_f1_score = 0
    # best_recall_score = 0
    # best_thresholds = []
    # best_threshold = 0

    # # for th in thresholds:
    # #     y_pred = predict(th, decision_scores)
    # #     f1 = f1_score(channel_data["label"], y_pred)
    # #     if (best_f1_score <= f1):
    # #         best_f1_score = f1
    # #         best_threshold = th

    # for th in thresholds:
    #     y_pred = predict(th, decision_scores)
    #     f1 = f1_score(channel_data["label"], y_pred)
    #     recall = recall_score(channel_data["label"], y_pred)
    #     if (best_recall_score <= recall):
    #         best_recall_score = recall
    #         best_thresholds.append(th) 
    
    # for th in best_thresholds:
    #     y_pred = predict(th, decision_scores)
    #     f1 = f1_score(channel_data["label"], y_pred)
    #     if (best_f1_score <= f1):
    #         best_f1_score = f1
    #         best_threshold = th
    
    y_test = channel_data["label"]
    # y_pred = predict(best_threshold, decision_scores)
    raw_y_pred = clf.predict(channel_data["test"])
    y_pred = []
    for y in raw_y_pred:
        if y == -1:
            y_pred.append(1)
        else:
            y_pred.append(0)
            
    # print("best_threshold: ", best_threshold)
    print(classification_report(y_test, y_pred))
    precision = precision_score(y_test, y_pred, average='weighted')
    print("FDR:", 1 - precision)
    c_m = confusion_matrix(y_test, y_pred)
    print(c_m)

In [33]:
def isolation_forest_manual(channel_data, channel):
    print("channel:", channel)
    #contamination=0
    clf = IsolationForest(contamination=0.5, random_state=42, n_jobs=-1).fit(channel_data["train"])
    decision_scores = clf.decision_function(channel_data["test"])
    
    thresholds = []
    for score in decision_scores: 
        if score < 0:
            thresholds.append(score)
    
    best_f1_score = 0
    best_recall_score = 0
    best_thresholds = []
    best_threshold = 0

    # for th in thresholds:
    #     y_pred = predict(th, decision_scores)
    #     f1 = f1_score(channel_data["label"], y_pred)
    #     if (best_f1_score <= f1):
    #         best_f1_score = f1
    #         best_threshold = th

    for th in thresholds:
        y_pred = predict(th, decision_scores)
        f1 = f1_score(channel_data["label"], y_pred)
        recall = recall_score(channel_data["label"], y_pred)
        if (best_recall_score <= recall):
            best_recall_score = recall
            best_thresholds.append(th) 
    
    for th in best_thresholds:
        y_pred = predict(th, decision_scores)
        f1 = f1_score(channel_data["label"], y_pred)
        if (best_f1_score <= f1):
            best_f1_score = f1
            best_threshold = th
    
    y_test = channel_data["label"]
    y_pred = predict(best_threshold, decision_scores)
    # raw_y_pred = clf.predict(channel_data["test"])
    # y_pred = []
    # for y in raw_y_pred:
    #     if y == -1:
    #         y_pred.append(1)
    #     else:
    #         y_pred.append(0)
            
    print("best_threshold: ", best_threshold)
    print(classification_report(y_test, y_pred))
    precision = precision_score(y_test, y_pred, average='weighted')
    print("FDR:", 1 - precision)
    c_m = confusion_matrix(y_test, y_pred)
    print(c_m)

In [ ]:
for channel in fitted_data:
    isolation_forest_manual(fitted_data[channel], channel)

channel: T
best_threshold:  -0.00013722055087511675
              precision    recall  f1-score   support

           0       0.89      0.56      0.69      1103
           1       0.19      0.60      0.29       189

    accuracy                           0.57      1292
   macro avg       0.54      0.58      0.49      1292
weighted avg       0.79      0.57      0.63      1292

FDR: 0.21047857950240767
[[620 483]
 [ 75 114]]
channel: G
best_threshold:  -0.00023553628446976171
              precision    recall  f1-score   support

           0       0.98      0.54      0.70      2354
           1       0.02      0.51      0.04        51

    accuracy                           0.54      2405
   macro avg       0.50      0.53      0.37      2405
weighted avg       0.96      0.54      0.68      2405

FDR: 0.03954476717361499
[[1274 1080]
 [  25   26]]
channel: S
best_threshold:  -0.017497584179967884
              precision    recall  f1-score   support

           0       0.96      0.69    

In [ ]:
for channel in fitted_data:
    isolation_forest_default(fitted_data[channel], channel)

In [ ]:
#implement KNN
